In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
60000/60000 [==============================] - 2s 33us/sample - loss: 0.2955 - acc: 0.9150
Epoch 2/5
60000/60000 [==============================] - 2s 32us/sample - loss: 0.1427 - acc: 0.9574
Epoch 3/5
60000/60000 [==============================] - 2s 32us/sample - loss: 0.1062 - acc: 0.9678
Epoch 4/5
60000/60000 [==============================] - 2s 32us/sample - loss: 0.0873 - acc: 0.9724
Epoch 5/5
10000/10000 [==============================] - 0s 18us/sample - loss: 0.0700 - acc: 0.9772


[0.07004223920041695, 0.9772]

In [71]:
_ , nrows , ncols = model.input_shape

def mkbod(lst):
    b = "\n".join(map(lambda x: "  ; " + x, lst))
    b = "  {" + b[3:] + "\n  }"
    return b

code = "type row = \n"
code += mkbod(["p{:02} : real".format(c) for c in range(ncols)])
code += "\n\n"

code += "type image = \n"
code += mkbod(["r{:02} : row".format(c) for c in range(nrows)])

code += "\n\n"
code += "type output = \n"
code += mkbod(["out{} : real".format(i) for i in range(10)])

typedecls = code

import itertools
input_flat_vector = \
    [ "x.r{:02}.p{:02}".format(r,c) 
     for r,c in itertools.product(range(nrows), range(ncols)) ]

In [72]:
print(typedecls)

type row = 
  { p00 : real
  ; p01 : real
  ; p02 : real
  ; p03 : real
  ; p04 : real
  ; p05 : real
  ; p06 : real
  ; p07 : real
  ; p08 : real
  ; p09 : real
  ; p10 : real
  ; p11 : real
  ; p12 : real
  ; p13 : real
  ; p14 : real
  ; p15 : real
  ; p16 : real
  ; p17 : real
  ; p18 : real
  ; p19 : real
  ; p20 : real
  ; p21 : real
  ; p22 : real
  ; p23 : real
  ; p24 : real
  ; p25 : real
  ; p26 : real
  ; p27 : real
  }

type image = 
  { r00 : row
  ; r01 : row
  ; r02 : row
  ; r03 : row
  ; r04 : row
  ; r05 : row
  ; r06 : row
  ; r07 : row
  ; r08 : row
  ; r09 : row
  ; r10 : row
  ; r11 : row
  ; r12 : row
  ; r13 : row
  ; r14 : row
  ; r15 : row
  ; r16 : row
  ; r17 : row
  ; r18 : row
  ; r19 : row
  ; r20 : row
  ; r21 : row
  ; r22 : row
  ; r23 : row
  ; r24 : row
  ; r25 : row
  ; r26 : row
  ; r27 : row
  }

type output = 
  { out0 : real
  ; out1 : real
  ; out2 : real
  ; out3 : real
  ; out4 : real
  ; out5 : real
  ; out6 : real
  ; out7 : real
  ; out8 

In [81]:
# obtain weights - generate iml
weights = model.get_weights()

code = ["let nn x = "]

# First layer
for j in range(weights[0].shape[1]):
    letin = []
    for i in range(weights[0].shape[0]):
        letin.append("({:.3}) * {}".format(weights[0][i][j],input_flat_vector[i]))
    letin.append("({:.3})".format(weights[1][j]))
    letin = "  let a{:03} = relu @@ ".format(j) + " + ".join(letin) + "  in"
    code.append(letin)

# Second layer
for j in range(weights[2].shape[1]):
    letin = []
    for i in range(weights[2].shape[0]):
        letin.append("({:.3}) * {}".format(weights[2][i][j],"a{:03}".format(i)))
    letin.append("({:.3})".format(weights[3][j]))
    letin = "  let out{} = ".format(j) + " + ".join(letin) + "  in"
    code.append(letin)
    
# Output
code.append("  { " + ";".join(["out{}".format(i) for i in range(10)]) + " }")


In [84]:
with open("nn.iml","w") as f:
    f.write("open Real")
    f.write("let relu x = if x > 0.0 then x else 0.0")
    
    f.write(typedecls)
    f.write("\n\n")
    f.write("\n".join(code))